# Atividade 2 - Visão Computacional

O entregável de toda esta atividade vai ser um código-fonte em *Python*. 

Encorajamos vocês a fazerem vídeos demonstrando o resultado e a postar (pode ser privadamente) no YouTube

Você pode entregar enviando o código para o Github e postando o vídeo *ou* mostrando ao vivo aos professores

**Não programe no Jupyter** - use um programa Python



Obs.: [Instale o GIT-FLS](../guides/git_lfs.md) para poder baixar este repositório.



Você deve ter uma folha com o padrão anexo. 
*Dica:* Se não tiver, é possível fazer também com um tablet ou *smartphone*
 
<img src="folha_atividade.png" width=300>

## Parte 1 - calibração

Ouça a explicação do professor sobre o modelo de câmera *pinhole*  e desenhe a medida $f$ que separa o plano focal da pupila da câmera

Detalhe como calculou $f$

In [2]:
h = 340.581869746

f = 30*h / 14
f 

729.8182923128571

## Parte 2

Modifique um dos exemplos `draw_circles_video.py` ou `videoplay.py` para passar a ler dados da webcam e identificar o círculo magenta e o círculo ciano, usando o `inRange`

In [4]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import cv2
import numpy as np
import time
import auxiliar as aux

cap = cv2.VideoCapture("masks.webm")

while(True):
    ret, frame = cap.read()
    
    if ret == False:
        print("Codigo de retorno FALSO - problema para capturar o frame")

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    #hsv1, hsv2 = aux.ranges("#516679")
    #hsv3 , hsv4 = aux.ranges("#95434f")
    hsv1 = np.array([150,  40,  50])
    hsv2 = np.array([180, 255, 255])
    
    hsv3 = np.array([85, 45, 45])
    hsv4 = np.array([110, 255, 255])
    
    mask0 = cv2.inRange(frame_hsv, hsv1, hsv2)   
    mask1 = cv2.inRange(frame_hsv, hsv3, hsv4)

    mask2 = mask0 + mask1

    cv2.imshow('img', mask2)

    if cv2.waitKey(9) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Codigo de retorno FALSO - problema para capturar o frame


error: OpenCV(4.1.1) /home/leviosa/404/scripts_robotica/OpenCV/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


## Parte 3

Assumindo que a folha se mantém sempre paralela ao plano de imagem da câmera, imprima a distância entre a folha e sua câmera

In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

__author__      = "Matheus Dib, Fabio de Miranda"


import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import auxiliar as aux
import math

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture("dist.webm")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

font = cv2.FONT_HERSHEY_SIMPLEX

print("Press q to QUIT")

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged
H = 14
h = 243.2
D = 30

f = D*h / H


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)

    circles = []


    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)

    if circles is not None:        
        circles = np.uint16(np.around(circles))
        lista_c = []

        for i in circles[0,:]:
            lista_c.append(i)
            if len(lista_c) == 2:
                x0 = int(lista_c[0][0])
                x1 = int(lista_c[1][0])
                y0 = int(lista_c[0][1])
                y1 = int(lista_c[1][1])
                
                xf = abs(x0 - x1)

                yf = abs(y0 - y1)
            

                if xf > 1000 or yf > 1000:
                    pass
                else:
                    pit = math.sqrt(xf**2 + yf**2)
                    dist = f*H/pit

                    if xf == 0:
                        cv2.putText(bordas_color,str(90)+'°',(100,100), font, 1,(255,255,255),2,cv2.LINE_AA)
                    else:
                        tang = float(yf)/float(xf)
                        seng = yf/dist
                        aang_rad = math.atan(tang)
                        aang_deg = math.degrees(aang_rad)
                        #print(tang,aang_deg,aang_rad,xf,yf)
                        #cv2.putText(bordas_color,str(aang_deg)+'°',(100,100), font, 1,(255,255,255),2,cv2.LINE_AA)
    
                cv2.putText(bordas_color,str(dist)+'cm',(0,50), font, 1,(255,255,255),2,cv2.LINE_AA)
                

                cv2.line(bordas_color,(x0,y0),(x1,y1),(255,0,0),5)

            # draw the outer circle

            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(255,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,255,0),3)

            
    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

#  When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


Press q to QUIT


## Parte 4

Trace uma linha entre os centros do círculo magenta e do círculo ciano.

Imprima na tela o ângulo entre esta linha e a horizontal

In [4]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

__author__      = "Matheus Dib, Fabio de Miranda"


import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import auxiliar as aux
import math

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture("angulo.webm")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

font = cv2.FONT_HERSHEY_SIMPLEX

print("Press q to QUIT")

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged
H = 14
h = 243.2
D = 30

f = D*h / H


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)

    circles = []


    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)

    if circles is not None:        
        circles = np.uint16(np.around(circles))
        lista_c = []

        for i in circles[0,:]:
            lista_c.append(i)
            if len(lista_c) == 2:
                x0 = int(lista_c[0][0])
                x1 = int(lista_c[1][0])
                y0 = int(lista_c[0][1])
                y1 = int(lista_c[1][1])
                
                xf = abs(x0 - x1)

                yf = abs(y0 - y1)
            

                if xf > 1000 or yf > 1000:
                    pass
                else:
                    pit = math.sqrt(xf**2 + yf**2)
                    dist = f*H/pit

                    if xf == 0:
                        cv2.putText(bordas_color,str(90)+'°',(100,100), font, 1,(255,255,255),2,cv2.LINE_AA)
                    else:
                        tang = float(yf)/float(xf)
                        seng = yf/dist
                        aang_rad = math.atan(tang)
                        aang_deg = math.degrees(aang_rad)
                        #print(tang,aang_deg,aang_rad,xf,yf)
                        cv2.putText(bordas_color,str(aang_deg)+'°',(0,50), font, 1,(255,255,255),2,cv2.LINE_AA)
    
               # cv2.putText(bordas_color,str(dist)+'cm',(0,50), font, 1,(255,255,255),2,cv2.LINE_AA)
                

                cv2.line(bordas_color,(x0,y0),(x1,y1),(255,0,0),5)

            # draw the outer circle

            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(255,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,255,0),3)

            

    # Draw a diagonal blue line with thickness of 5 px
    # cv2.line(img, pt1, pt2, color[, thickness[, lineType[, shift]]])
    cv2.line(bordas_color,(0,0),(511,511),(255,0,0),5)

    # cv2.rectangle(img, pt1, pt2, color[, thickness[, lineType[, shift]]])
    cv2.rectangle(bordas_color,(384,0),(510,128),(0,255,0),3)

    # cv2.putText(img, text, org, fontFace, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
    # font = cv2.FONT_HERSHEY_SIMPLEX
    # cv2.putText(bordas_color,'Press q to quit',(0,50), font, 1,(255,255,255),2,cv2.LINE_AA)

    #More drawing functions @ http://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html

    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

#  When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


Press q to QUIT


## Parte 5

Baseado nos círculos encontrados pela transformada de Hough, desenhe um círculo sobre o círculo ciano e outro sobre o círculo magenta.

Dica: Use a função `cv2.circle` e veja o exemplo [draw_circles_video.py](draw_circles_video.py).

**Desafio bônus**: ser capaz de eliminar circulos espúrios (aqueles que não são os da folha)

## Parte 6

Usando um detector de features `BRISK`, identifique o escrito *Insper* na folha. Veja os exemplos [no notebook](BRISK%20Features.ipynb) e em [em código](features.py) .

In [1]:
# Essa função vai ser usada abaixo. Ela encontra a matriz (homografia) 
# que quando multiplicada pela imagem de entrada gera a imagem de 


def find_homography_draw_box(kp1, kp2, img_cena):
    
    out = img_cena.copy()
    
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)


    # Tenta achar uma trasformacao composta de rotacao, translacao e escala que situe uma imagem na outra
    # Esta transformação é chamada de homografia 
    # Para saber mais veja 
    # https://docs.opencv.org/3.4/d9/dab/tutorial_homography.html
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()


    
    h,w = img_original.shape
    # Um retângulo com as dimensões da imagem original
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

    # Transforma os pontos do retângulo para onde estao na imagem destino usando a homografia encontrada
    dst = cv2.perspectiveTransform(pts,M)


    # Desenha um contorno em vermelho ao redor de onde o objeto foi encontrado
    img2b = cv2.polylines(out,[np.int32(dst)],True,(255,255,0),5, cv2.LINE_AA)
    
    return img2b
    